$\beta = -1$ means that $\beta$ is selected at random in each iteration.

In [15]:
from grasp_betas import read_results

betas = read_results()

In [16]:
betas.groupby("beta").mean()

,OF,time,improvs
beta,,,
-1.00,261.355556,44.086735,2.222222
0.00,261.922222,36.081349,2.233333
0.25,261.511111,39.455396,2.233333
0.50,260.833333,42.723133,2.222222
0.75,261.011111,50.015056,2.444444
1.00,261.122222,57.962084,2.288889


In [21]:
betas.loc[500, 500, 125, 2].mean()

OF          91.100000
time       165.560893
improvs      2.333333
dtype: float64